This is a notebook that I will use to define the functions to automate scraping from Nepremicnine.net. 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:
import numpy as np
import pandas as pd
import bs4
import lxml.etree as xml
import time

Define path to the webdriver and define webdriver as driver. 

In [5]:
PATH = '/Users/Asus/Downloads/chromedriver' # Defines where my cromedriver package is installed. Opens an automated Chrome window.
driver = webdriver.Chrome(PATH) # names webdriver as driver and assigns path. 
driver.quit() # closes the driver so the window is not always on. 

In [7]:
# give base url to the site I want to scrape.
url = 'http://nepremicnine.net'

In [9]:
driver = webdriver.Chrome(PATH) 
driver.get(url) #automatically opens Nepremicnine.net